In [73]:
import pandas as pd
import requests
import lxml.html as lh
import geopy as gp

pd.options.display.max_colwidth = 150

In [2]:
store_list = []
for page in range(28):
    stores = requests.get("https://api.parknshop.com/api/v2/pnshk/stores/watStores?currentPage="+str(page)+"&pageSize=10&isCceOrCc=false&fields=FULL&lang=en_HK&curr=HKD")\
        .json()["stores"]
    for node in stores:
        store_list.append({'address': node["address"]["displayAddress1"],
                            'district': node["address"]["district"],
                            'town': node["address"]["town"],
                            'city': node["address"]["county"],
                            'lat' : node["geoPoint"]["latitude"],
                            'lon' : node["geoPoint"]["longitude"],
                          })

parknshop = pd.DataFrame(store_list, columns = ['address', 'district', 'town', 'city', 'lat', 'lon'])

In [3]:
parknshop.head(3)

,address,district,town,city,lat,lon
0,"G/F, Best-O-Best Commercial Centre, 32-36 Ferry Street, Yau Ma Tei, Kowloon",Yau Ma Tei,Kowloon,Hong Kong,22.307602,114.167647
1,"Shop N50B, G/F, Mount Sterling Mall, Stage 7 Mei Foo Sun Chuen, Nos.10-16 Lai Wan Road, Lai Chi ...",Mei Foo,Kowloon,Hong Kong,22.337786,114.138378
2,"Fine Mansion, 32-40 Village Road, Happy Valley, Hong Kong",Happy Valley,Hong Kong Island,Hong Kong,22.268610,114.183627


In [4]:
url = requests.get("https://www.wellcome.com.hk/en/our-store")
doc = lh.fromstring(url.content) #Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

col = []
i = 0

In [5]:
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [51]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]

    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
Dict={title:column for (title,column) in col}
wellcome=pd.DataFrame(Dict)

In [7]:
wellcome.head()

,Store,Address,Telephone,Opening Hours
0,Tsuen Wan,"57-61, Lo Tak Court, G/F, Tsuen Wan, NT",2411 6796,07:00-24:00
1,Pak Tin Estate,"Shop LG201, Lower Ground Level 2, Pak Tin Commercial Centre, Pak Tin Estate, Kowloon",2335 9738,08:00-22:00
2,Tak Po Garden 2,"Shop 138, G/F, TBG Mall, Tak Bo Garden, No. 3 Ngau Tau Kok Road, Kowloon, Hong Kong",2382 3453,08:00-22:00
3,Dor Hei Building,"Shop No.2-3, G/F, Dor Hei Building, Nos.9-17 Ting Fu Street, Nos.86-88 Ting On Street, Ngau Tau ...",2628 4308,08:00-22:00
4,Chevalier House,"Shop C and Portion of Shop D on Ground Floor, Chevalier House, Nos. 45-51 Chatham Road South, Ko...",2713 2178,08:00–22:00


In [8]:
#Clean

In [9]:
parknshop['district'] = parknshop['district'].replace('mawan', 'Ma Wan')

In [10]:
parknshop.query('district == "Ma Wan"')

,address,district,town,city,lat,lon
212,"Shop No. S7, Level 1, Park Island, Ma Wan Island, New Territories",Ma Wan,New Territories,Hong Kong,22.350473,114.061296


In [11]:
parknshop = parknshop.drop_duplicates()

In [52]:
wellcome = wellcome.drop_duplicates()

In [53]:
wellcome['town'] = wellcome['Address'].apply(lambda x: x.split(',')[-1])

In [54]:
wellcome['temp_district'] = ''
for x in range(len(wellcome['Address'])):
    dis = len(wellcome['Address'][x].split(','))-2
    wellcome['temp_district'][x] = wellcome['Address'][x].split(',')[dis]

In [55]:
wellcome['town'] = wellcome['town'].str.strip()

In [56]:
wellcome['temp_district'] = wellcome['temp_district'].str.strip()

In [57]:
wellcome['Store'] = wellcome['Store'].str.strip()

In [58]:
wellcome['Store'] = wellcome['Store'].str.replace('\d+', '')

In [59]:
wellcome['Address'] = wellcome['Address'].str.strip()

In [60]:
wellcome['Address'] = wellcome['Address'].str.replace('Shop No.25 G/F, Victoria Centre, No.15 Watson Road, North Point', 'Shop No.25 G/F, Victoria Centre, 15 Watson Road, North Point')

In [61]:
wellcome['town'] = wellcome['town'].str.replace('H.*', 'Hong Kong')
wellcome['town'] = wellcome['town'].str.replace('Central', 'Hong Kong')
wellcome['town'] = wellcome['town'].str.replace('Kennedy Town.', 'Hong Kong')
wellcome['town'] = wellcome['town'].str.replace('North Point', 'Hong Kong')
wellcome['town'] = wellcome['town'].str.replace('Shau Kei Wan', 'Hong Kong')

In [62]:
wellcome['town'] = wellcome['town'].str.replace('N.T.*', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('NT.*', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('NT.', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('New Territories.', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('NEW TERRITORIES', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Sha Tau Kok', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Tai Po', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('.*Chau.*', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Lantau Island', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Tsing Yi', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Tsuen Wan', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Tuen Mun.*', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Kwai Chung', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Wu Kai Sha', 'New Territories')
wellcome['town'] = wellcome['town'].str.replace('Yuen Long', 'New Territories')

In [63]:
wellcome['town'] = wellcome['town'].str.replace('KLN', 'Kowloon')
wellcome['town'] = wellcome['town'].str.replace('Kln.*', 'Kowloon')
wellcome['town'] = wellcome['town'].str.replace('Kowloon.', 'Kowloon')
wellcome['town'] = wellcome['town'].str.replace('No. 204-206 Nathan Road. Kowloon', 'Kowloon')
wellcome['town'] = wellcome['town'].str.replace('Shop 3 & 4 G/F Cite 33 No 33 Lai Chi Kok Road Kowloon', 'Kowloon')
wellcome['town'] = wellcome['town'].str.replace('Tokwawan', 'Kowloon')

In [ ]:
for x in range(len(wellcome)):
    if (wellcome['town'][x] == 'Hong Kong' and wellcome['temp_district'][x] == 'Kowloon'):
        wellcome['town'][x] = wellcome['temp_district'][x]

In [101]:
from geopy.extra.rate_limiter import RateLimiter

geolocator = gp.Nominatim(user_agent="myapp")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

wellcome['temp_add'] = wellcome['Store']+", "+wellcome['town']

wellcome['location'] = wellcome['temp_add'].apply(geocode)
wellcome['lat'] = wellcome['location'].apply(lambda x: x.latitude if x else None)
wellcome['lon'] = wellcome['location'].apply(lambda x: x.longitude if x else None)

wellcome['location'] = wellcome['location'].astype(str)

wellcome

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Park Vale, Hong Kong',), **{}).
Traceback (most recent call last):
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 1369, in getresponse
    response.begin()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Pr

RateLimiter swallowed an error after 2 retries. Called with (*('Park Vale, Hong Kong',), **{}).
Traceback (most recent call last):
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 1369, in getresponse
    response.begin()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\http\client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Progr

,Store,Address,Telephone,Opening Hours,town,temp_district,temp_add,location,lat,lon,te
0,Tsuen Wan,"57-61, Lo Tak Court, G/F, Tsuen Wan, NT",2411 6796,07:00-24:00,New Territories,Tsuen Wan,"Tsuen Wan, New Territories","荃灣 Tsuen Wan, 荃灣區 Tsuen Wan District, 新界 New Territories, 香港 Hong Kong, 中国",22.371661,114.113470,荃灣區 Tsuen Wan District
1,Pak Tin Estate,"Shop LG201, Lower Ground Level 2, Pak Tin Commercial Centre, Pak Tin Estate, Kowloon",2335 9738,08:00-22:00,Kowloon,Pak Tin Estate,"Pak Tin Estate, Kowloon","白田邨 Pak Tin Estate, 石硤尾 Shek Kip Mei, 深水埗區 Sham Shui Po District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.336663,114.167010,深水埗區 Sham Shui Po District
2,Tak Po Garden,"Shop 138, G/F, TBG Mall, Tak Bo Garden, No. 3 Ngau Tau Kok Road, Kowloon, Hong Kong",2382 3453,08:00-22:00,Kowloon,Kowloon,"Tak Po Garden , Kowloon","德雅中學 Tak Nga Secondary School, 達之路 Tat Chee Avenue, 秀蘭苑 Orchid Garden, 石硤尾 Shek Kip Mei, 深水埗區 Sham Shui Po District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.330126,114.175275,深水埗區 Sham Shui Po District
3,Dor Hei Building,"Shop No.2-3, G/F, Dor Hei Building, Nos.9-17 Ting Fu Street, Nos.86-88 Ting On Street, Ngau Tau Kok, Kowloon",2628 4308,08:00-22:00,Kowloon,Ngau Tau Kok,"Dor Hei Building, Kowloon","多喜大廈 Dor Hei Building, 牛頭角 Ngau Tau Kok, 觀塘區 Kwun Tong District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.317369,114.216320,觀塘區 Kwun Tong District
4,Chevalier House,"Shop C and Portion of Shop D on Ground Floor, Chevalier House, Nos. 45-51 Chatham Road South, Kowloon",2713 2178,08:00–22:00,Kowloon,Nos. 45-51 Chatham Road South,"Chevalier House, Kowloon","其士大廈 Chevalier House, 尖沙咀 Tsim Sha Tsui, 油尖旺區 Yau Tsim Mong District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.298987,114.175525,油尖旺區 Yau Tsim Mong District
...,...,...,...,...,...,...,...,...,...,...,...
274,Nina Mall,"Shop No. 222 on Level 2 of Nina Mall l, No.8 Yeung Uk Road, Tsuen Wan, N.T",2422 2371,08:30-22:00,New Territories,Tsuen Wan,"Nina Mall, New Territories","如心廣場商場 I Nina Mall I, 楊屋道 Yeung Uk Road, 荃灣東工業區 Tsuen Wan East Industrial Area, 荃灣市中心 Tsuen Wan Town Centre, 荃灣 Tsuen Wan, 荃灣區 Tsuen Wan District,...",22.368376,114.112917,
275,Nam Cheong Street,"Shop 102, 223-237 Nam Cheong Street, Kowloon",2779 2515,07:30–23:00,Kowloon,223-237 Nam Cheong Street,"Nam Cheong Street, Kowloon","南昌街 Nam Cheong Street, 大角咀 Tai Kok Tsui, 油尖旺區 Yau Tsim Mong District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.326144,114.160806,
276,Sceneway Plaza,"Shop 2F Level 4, Sceneway Garden, No. 8 Sceneway Road, Lam Tin, Kowloon",2205 6532,08:00-22:30,Kowloon,Lam Tin,"Sceneway Plaza, Kowloon","匯景廣場 Sceneway Plaza, 啟田道 Kai Tin Road, 藍田 Lam Tin, 觀塘區 Kwun Tong District, 九龍 Kowloon, 香港 Hong Kong, 中国",22.307044,114.233931,
277,Sau Mau Ping Plaza,"Shop 102, 1/F Sau Mau Ping Shopping Centre, Sau Mau Ping Estate, Kwun Tong, KLN",2354 8534,08:00-23:00,Kowloon,Kwun Tong,"Sau Mau Ping Plaza, Kowloon",nan,NaN,NaN,


In [104]:
wellcome['district']=""
for x in range(len(wellcome['location'])):
    dis = (len(wellcome['location'][x].split(',')))-4
    if dis > 0:
        wellcome['district'][x] = wellcome['location'][x].split(',')[dis]

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [106]:
for x in range(len(wellcome['location'])):
    dis = (len(wellcome['location'][x].split(',')))-3
    if dis > 0:
        wellcome['town'][x] = wellcome['location'][x].split(',')[dis]

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [112]:
wellcome = wellcome.drop(['temp_district', 'temp_add', 'Telephone', 'Opening Hours'], axis=1)